In [1]:
import requests
import json
import popelines
import os
from datetime import datetime


def fix_values(value, key, reset_key):
    if key == reset_key:
        new_list = []
        for x in value:
            value[x][f'{reset_key}_id'] = x
            new_list.append(value[x])
        return new_list
    else:
        return value

In [2]:
def flatten(jayson, acc, prefix):
    if isinstance(jayson, dict):
        for k,v in jayson.items():
            if prefix:
                prefix_k = prefix + "_" + k
            else: 
                prefix_k = k
            prefix_k = prefix_k.replace('-', '_')
            
            if isinstance(v, dict):
                flatten(v, acc, prefix_k)
            elif isinstance(v, list):
                for j in v:
                    flatten(j, acc, prefix_k)
            else:
                acc[prefix_k] = v
        return acc 
    else:
        return acc

In [3]:
def generate_project_experiment(project_endpoint, experiment_endpoint):

    # get all projects
    try:
        response_proj = requests.get(project_endpoint, headers=headers)
        j_proj = json.loads(response_proj.text)
        response_proj.raise_for_status()
        print('Successfully read project data.')

    except requests.exceptions.HTTPError as err:
        print(err)

    # store a list of project metadata
    project_list = []
    experiment_id_list = []

    for project in j_proj:
        p_id = project['id']

        # get all experiments from one project
        params = (
            ('project_id', p_id),
            ('per_page', 100),
        ) 

        try:
            response_exp = requests.get(experiment_endpoint, headers=headers, params=params)
            j_exp = json.loads(response_exp.text)
            response_exp.raise_for_status()
            print('Successfully read experiment data.')
        except requests.exceptions.HTTPError as err:
            print(err)

        # loop for all experiments in this project
        upload_exp_list = []
        for exp in j_exp:
            exp['project_id'] = p_id
            upload_exp_list.append(flatten(exp, {}, ''))
            experiment_id_list.append(exp['id'])
            
        # upload experiment 
        # pope.write_to_json(file_name='../uploads/experiments.json', jayson=upload_exp_list, mode='w')
        # pope.write_to_bq(table_name='experiments', file_name='../uploads/experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
        # print(f"Successfully uploaded experiments for project {p_id}")

    project_list.append(project)

    # pope.write_to_json(file_name='../uploads/projects.json', jayson=project_list, mode='w')
    # pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
    # print("Successfully uploaded all projects.")

    return experiment_id_list

In [4]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

############################################### Project-Experiment #############################################
p_endpoint = 'https://api.optimizely.com/v2/projects'
e_endpoint = 'https://api.optimizely.com/v2/experiments'

experiment_id = generate_project_experiment(project_endpoint=p_endpoint, experiment_endpoint=e_endpoint)


Successfully read project data.
Successfully read experiment data.
Successfully read experiment data.
Successfully read experiment data.
Successfully read experiment data.
Successfully read experiment data.


In [5]:
all_ts = []
############################################### Experiment-Results #############################################
for exp_id in experiment_id:
    print(exp_id)
    ts_endpoint = f'https://api.optimizely.com/v2/experiments/{exp_id}/timeseries'
    # generate_experiment_results(ts_endpoint, experiment_id=exp_id)

    response_ts = requests.get(ts_endpoint, headers=headers)
    if response_ts.text == '':
        # if '' then the experiment has not started yet
        j_ts = {'experiment_id': exp_id}
    else:
        j_ts = json.loads(response_ts.text)

    new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')
    # new_j_ts_list = [new_j_ts]

    all_ts.append(new_j_ts)

10756790433
10796678057
10800885728
10815382176
10945903729
10955300529
11039523400
11938601024
11966791385
11992110033
10488931082
10545851126
10556110251
10564323775
10605041742
10622090898
10666463868
10675031376
10681551672
10683631399
10715090552
10733622163
10797220881
10797550884
10805332324
10844011372
10854082584
10882272812
11011180459
11066932017
11085711684
11087580674
11174107321
11189744252
11191727326
11191905720
11199410199
11206301430
11308083317
11334340375
11356331084
11367191147
11378763890
11396076650
11404885421
11406773739
11432181239
11474010139
11482931694
11556750451
11562826019
11570380944
11657860282
11713452853
11720710615
11722600722
12016386767
12096030597
12117040288
12132980378
12166698246
12169545706
12239002639
12240020740
12275901710
12337280003
12850290083
12903040293
13026380185
13026390548
11377370874
12107582952
12632940119
12722820205
12743540432
12746930280
10702922345
10731441129
10758540726
10798551869
10799420505
10801250617
10801721220
1080

In [6]:
len(all_ts)

108

In [7]:
os.system('say "completed"')

0

In [34]:
def generate_experiment_results(ts_endpoint):
    # get result time series from all experiments:
    try:
        response_ts = requests.get(ts_endpoint, headers=headers)
        j_ts = json.loads(response_ts.text)
        response_ts.raise_for_status()
        print('Successfully read experiment results data.')
    except requests.exceptions.HTTPError as err:
        print(err)
    
    # with keys properly reset, we need to populate upper level into each row of each list level
    # Trying to flatten inner level 'timeseries'
    flattened_metrics = []
    for metric in new_j_ts['metrics']:
        for ts in metric['results']:
            flattened_timeseries = []
            for element in ts['timeseries']:
                flattened_timeseries.append(flatten(element, {}, ''))

            # Replace old 'timeseries' with new 'flattened_timeseries'
            updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
            flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')

        # Replace old 'metrics' with new 'flattened_results'
        update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
        flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))
        
    return flattened_metrics

In [36]:
def populating_vals(outer_dict, inner_flattened_list, destination_key):
    # outer_dict is the upper level dict
    # inner_flattened_list is the lower level list of dicts that are each flattened
    # destination_key is the primary key of the lower level
    updated_dict = {}
    for k, v in outer_dict.items():
        if k != destination_key:
            updated_dict[k] = v
        else:
            updated_dict[k] = inner_flattened_list
    return updated_dict

In [38]:
def flatten_dupe_vals(vals, key):
    
    duped_results = []
    for element in vals[key]:
        ts_dict = populating_vals(outer_dict=vals, inner_flattened_list=element, destination_key=key)
        duped_results.append(ts_dict)

    flattened_results = []
    for element in duped_results:
        flattened_results.append(flatten(element, {}, ''))
        
    return flattened_results

In [40]:
ts = generate_experiment_results(ts_endpoint='https://api.optimizely.com/v2/experiments/11402551541/timeseries')


Successfully read experiment results data.


In [82]:
for e in ts:
    print(e['results_name'])

Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (Control)
Free Guide (C

In [44]:
!pwd

/Users/vicky/Dev/optimizely/scripts


In [45]:
!ls

generate_original.py  scratch_keap.ipynb    test_results_all.json


In [46]:
pope.write_to_json(file_name='results_ts.json', jayson=ts, mode='w')

In [47]:
!ls

generate_original.py  scratch_keap.ipynb
results_ts.json       test_results_all.json


In [58]:
pope.write_to_bq(table_name='result_ts', file_name='results_ts.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)


INFO:root:Writing result_ts to BQ from file results_ts.json


2019-01-29 15:08:30,352 - root - Writing result_ts to BQ from file results_ts.json


# experiments

In [60]:
# # get all projects
# try:
#     response_proj = requests.get('https://api.optimizely.com/v2/projects', headers=headers)
#     j_proj = json.loads(response_proj.text)
#     response_proj.raise_for_status()
#     print('Successfully read project data.')

# except requests.exceptions.HTTPError as err:
#     print(err)

# # store a list of project metadata
# project_list = []
experiment_id_list = []

# for project in j_proj:
    p_id = project['id']

    # get all experiments from one project
    params = (
        ('project_id', p_id),
        ('per_page', 100),
    ) 

    try:
        response_exp = requests.get('https://api.optimizely.com/v2/experiments', headers=headers, params=params)
        j_exp = json.loads(response_exp.text)
        response_exp.raise_for_status()
        print('Successfully read experiment data.')
    except requests.exceptions.HTTPError as err:
        print(err)

    # loop for all experiments in this project
    upload_exp_list = []
    for exp in j_exp:
        exp['project_id'] = p_id
        upload_exp_list.append(flatten(exp, {}, ''))
        experiment_id_list.append(exp['id'])

    # upload experiment 
    # pope.write_to_json(file_name='../uploads/experiments.json', jayson=upload_exp_list, mode='w')
    # pope.write_to_bq(table_name='experiments', file_name='../uploads/experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
    # print(f"Successfully uploaded experiments for project {p_id}")

# project_list.append(project)

# pope.write_to_json(file_name='../uploads/projects.json', jayson=project_list, mode='w')
# pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
# print("Successfully uploaded all projects.")


Successfully read project data.
Successfully read experiment data.
Successfully read experiment data.
Successfully read experiment data.
Successfully read experiment data.
Successfully read experiment data.


In [68]:
p_id = 10849384554

# get all experiments from one project
params = (
    ('project_id', p_id),
    ('per_page', 100),
) 

try:
    response_exp = requests.get('https://api.optimizely.com/v2/experiments', headers=headers, params=params)
    j_exp = json.loads(response_exp.text)
    response_exp.raise_for_status()
    print('Successfully read experiment data.')
except requests.exceptions.HTTPError as err:
    print(err)

# loop for all experiments in this project
upload_exp_list = []
for exp in j_exp:
    exp['project_id'] = p_id
    upload_exp_list.append(exp)
#     upload_exp_list.append(flatten(exp, {}, ''))
#     experiment_id_list.append(exp['id'])

Successfully read experiment data.


In [80]:
upload_exp_list[0]

{'allocation_policy': 'manual',
 'audience_conditions': 'everyone',
 'campaign_id': 10880750271,
 'changes': [],
 'created': '2018-06-19T21:14:50.593989Z',
 'description': 'That existing copy "Get it together," "Make it personal," & "Cash it in" is more effective than the Close more business version. ',
 'earliest': '2018-06-22T16:34:45.276504Z',
 'holdback': 0,
 'id': 10898650266,
 'is_classic': False,
 'last_modified': '2018-11-08T20:25:59.779273Z',
 'latest': '2018-11-08T20:26:00.704897Z',
 'metrics': [{'aggregator': 'unique',
   'event_id': 10894182018,
   'scope': 'visitor',
   'winning_direction': 'increasing'}],
 'name': "Homepage - Close More Business CTA - Jun '18",
 'project_id': 10849384554,
 'status': 'paused',
 'type': 'a/b',
 'url_targeting': {'activation_type': 'immediate',
  'conditions': '["and", ["or", {"match_type": "simple", "type": "url", "value": "https://www.infusionsoft.com/"}]]',
  'edit_url': 'https://www.infusionsoft.com/',
  'key': '10849384554_url_targeting

In [69]:
for e in upload_exp_list:
    print(e['variations_variation_id'])

KeyError: 'variations_variation_id'

In [79]:
pope.write_to_json(file_name='experiments.json', jayson=upload_exp_list, mode='w')
pope.write_to_bq(table_name='test_experiments', file_name='experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
    
    
    

INFO:root:Writing test_experiments to BQ from file experiments.json


2019-01-29 15:49:34,393 - root - Writing test_experiments to BQ from file experiments.json


In [ ]:
11396315769
11365566229
11388339167